In [1]:
!pip install pypdf sentence-transformers qdrant-client tqdm python-dotenv

  Using cached sentence_transformers-5.1.2-py3-none-any.whl.metadata (16 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached scikit_learn-1.7.2-cp311-cp311-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached scipy-1.16.3-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pillow-12.0.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (8.8 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached numpy-2.3.5-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pyyaml-6.0.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (2.4 kB)
  Using cached regex-2025.11.3-cp311-cp311-macosx_11_0_arm64.whl.metadata (40 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached fsspec

In [ ]:
import os
import hashlib
from pypdf import PdfReader
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient, models
from tqdm import tqdm
import os 
from dotenv import load_dotenv

load_dotenv()



PDF_DIR = "downloaded"
COLLECTION_NAME = "astro_physics_papers"
QDRANT_URL = "YOUR_QDRANT_URL_HERE"
QDRANT_KEY = "YOUR_QDRANT_API_KEY_HERE"
EMBEDDING_MODEL = "sentence-transformers/all-MiniLM-L6-v2"

# Load model
try:
    model = SentenceTransformer(EMBEDDING_MODEL, device="gpu")
    print("Using GPU")
except:
    model = SentenceTransformer(EMBEDDING_MODEL)
    print("Using CPU")

qdrant = QdrantClient(url=QDRANT_URL
                       , api_key=QDRANT_KEY)



if COLLECTION_NAME not in [d.name for d in qdrant.get_collections().collections]:
    qdrant.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=models.VectorParams(
            size=model.get_sentence_embedding_dimension(),
            distance=models.Distance.COSINE,
        )
    )

def extract_pages(pdf_path):
    reader = PdfReader(pdf_path)
    return [(i+1, page.extract_text() or "") for i, page in enumerate(reader.pages)]

def deterministic_id(filename, page_number):
    key = f"{filename}-{page_number}".encode()
    return int(hashlib.sha256(key).hexdigest(), 16) % (2**63)

# --------------- Batching setup ---------------
BATCH_SIZE = 64
buffer = []

def flush_buffer():
    if not buffer:
        return
    qdrant.upsert(
        collection_name=COLLECTION_NAME,
        points=[
            models.PointStruct(id=pid, vector=vec, payload=pay)
            for pid, vec, pay in buffer
        ]
    )
    buffer.clear()


# ----------------------------------------------
# PROCESS ALL PDFs
# ----------------------------------------------

for filename in tqdm(os.listdir(PDF_DIR)):
    if not filename.lower().endswith(".pdf"):
        continue

    path = os.path.join(PDF_DIR, filename)
    print(f"Processing: {filename}")

    pages = extract_pages(path)

    texts = []
    ids = []
    payloads = []

    for page_number, text in pages:
        if not text.strip():
            continue

        pid = deterministic_id(filename, page_number)

        ids.append(pid)
        texts.append(text)
        payloads.append({
            "document": filename,
            "page": page_number,
            "text": text
        })

        # When batch locally reaches 64 items
        if len(texts) == BATCH_SIZE:
            embeddings = model.encode(texts).tolist()
            for pid, emb, pay in zip(ids, embeddings, payloads):
                buffer.append((pid, emb, pay))
            flush_buffer()

            texts, ids, payloads = [], [], []

    # leftover pages
    if texts:
        embeddings = model.encode(texts).tolist()
        for pid, emb, pay in zip(ids, embeddings, payloads):
            buffer.append((pid, emb, pay))
        flush_buffer()

flush_buffer()
print("DONE — All documents uploaded.")
